# Insurance Premium Calculator
(using Chain Ladder method for loss development)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
import pandas as pd
import numpy as np

Import CSV file with 5-years of loss development information (claim counts)

In [3]:
data = pd.read_csv("sample_claim_data.csv")

In [4]:
data = data.iloc[:,1:]

In [5]:
data.fillna("")

,12_mos,24_mos,36_mos,48_mos,60_mos,72_mos
0,242,384,496,496,496,
1,278,366,467,467,,
2,310,412,532,,,
3,434,592,,,,
4,483,,,,,


Develop Age to Age Loss Development factors

In [6]:
age_to_age = {}
for i in range(len(data.columns)-1):
    age_to_age[str(data.columns[i] + "->" + data.columns[i+1])] = data.iloc[:,i+1] / data.iloc[:,i]

In [7]:
loss_dev_factors = pd.DataFrame(age_to_age)

In [8]:
loss_dev_factors.iloc[0:-1,:].fillna("").replace([0, 0.0],"")

,12_mos->24_mos,24_mos->36_mos,36_mos->48_mos,48_mos->60_mos,60_mos->72_mos
0,1.586777,1.29167,1,1,
1,1.316547,1.27596,1,,
2,1.329032,1.29126,,,
3,1.364055,,,,


Calculate average and weighted average factors

In [9]:
avg_and_wavg_dict = {}

for i in range(len(loss_dev_factors.columns)):
    
    avg_and_wavg_dict[str(loss_dev_factors.columns[i])] = [np.mean(loss_dev_factors.iloc[:,i]),np.sum(loss_dev_factors.iloc[:,i] / np.sum(loss_dev_factors.iloc[:,i]) * loss_dev_factors.iloc[:,i])]

In [10]:
avg_and_wavg = pd.DataFrame(avg_and_wavg_dict)

In [11]:
avg_and_wavg.fillna("").replace([0, 0.0],"")

,12_mos->24_mos,24_mos->36_mos,36_mos->48_mos,48_mos->60_mos,60_mos->72_mos
0,1.399103,1.286295,1.0,1.0,
1,1.407711,1.286337,1.0,1.0,


Select loss dev factors to be used 

In [12]:
selection = 0

In [13]:
if selection == 0:
    sel_factors = avg_and_wavg.iloc[0,:]
    sel_factors.fillna(1.00, inplace=True)
    sel_factors.replace([0,0.0],1.00,inplace=True)
    sel_factors = pd.DataFrame(sel_factors)
    sel_factors = sel_factors.transpose()
elif selection == 1:
    sel_factors = avg_and_wavg.iloc[1,:]
    sel_factors.fillna(1.00, inplace=True)
    sel_factors.replace([0,0.0],1.00,inplace=True)
    sel_factors = pd.DataFrame(sel_factors)
    sel_factors = sel_factors.transpose()

In [14]:
sel_factors

,12_mos->24_mos,24_mos->36_mos,36_mos->48_mos,48_mos->60_mos,60_mos->72_mos
0,1.399103,1.286295,1.0,1.0,1.0


Calculate cumulative factors (to ultimate)

In [15]:
cumulative = {}
for i in range(len(sel_factors.columns)):
    cumulative[str(sel_factors.columns[i].split("->")[0] + "_to_Ult")] = [np.prod(sel_factors.iloc[0,i:])]

In [16]:
cumulative_factors = pd.DataFrame(cumulative)

In [17]:
cumulative_factors

,12_mos_to_Ult,24_mos_to_Ult,36_mos_to_Ult,48_mos_to_Ult,60_mos_to_Ult
0,1.799659,1.286295,1.0,1.0,1.0


Import CSV file with additional information (policy count/total claims)

In [18]:
data_2 = pd.read_csv("sample_addtl_info.csv")

In [19]:
data_2

,Year,pol_count,tot_clm_amt
0,1,8613,5028600
1,2,12712,5851530
2,3,16446,9830106
3,4,17452,10362368
4,5,18812,10506240


Calculate indicated ultimate, frequency, and severity (average claim amount)

In [20]:
freq_sev_dict = {"maturity":[], "clm_count":[], "sel_fact_to_ult":[], "ind_ult":[], "pol_count":[], "frequency":[], "severity":[]}
            
for i in range(len(data.columns)-1):
    freq_sev_dict["maturity"].append(data.columns[-2-i])
    freq_sev_dict["clm_count"].append(data.iloc[i,-2-i])
    freq_sev_dict["sel_fact_to_ult"].append(cumulative_factors.iloc[0,-1-i])
    freq_sev_dict["ind_ult"].append(freq_sev_dict["clm_count"][i]*freq_sev_dict["sel_fact_to_ult"][i])
    freq_sev_dict["pol_count"].append(data_2.iloc[i,1])
    freq_sev_dict["frequency"].append(freq_sev_dict["ind_ult"][i]/freq_sev_dict["pol_count"][i])
    freq_sev_dict["severity"].append(data_2.iloc[i,2]/freq_sev_dict["clm_count"][i])

In [21]:
freq_sev = pd.DataFrame(freq_sev_dict)

In [22]:
freq_sev

,maturity,clm_count,sel_fact_to_ult,ind_ult,pol_count,frequency,severity
0,60_mos,496.0,1.000000,496.000000,8613,0.057587,10138.306452
1,48_mos,467.0,1.000000,467.000000,12712,0.036737,12530.042827
2,36_mos,532.0,1.000000,532.000000,16446,0.032348,18477.642857
3,24_mos,592.0,1.286295,761.486657,17452,0.043633,17504.000000
4,12_mos,483.0,1.799659,869.235283,18812,0.046206,21752.049689


Calculate Loss Cost (using 5-year average frequency and severity)

In [23]:
loss_cost = np.mean(freq_sev["frequency"] * np.mean(freq_sev["severity"]))

In [24]:
loss_cost

696.3210133027999

Input Underwriting Profit and Underwriting Expenses (as % of Premium)

In [28]:
uw_profit = 0.10
uw_expense = 0.15

Calculate Final Premium

In [29]:
premium = loss_cost * (1/(1-uw_profit)) * (1/(1-uw_expense))

In [32]:
print("$",round(premium,2),sep="")

$910.22
